In [1]:
import numpy as np
import os
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import MinMaxScaler, RobustScaler, scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import modules.feature_extraction as fe

#### Features aus csv lesen

In [2]:
df = pd.read_csv(r"data\features.csv", sep=';', header=None)
df = df.iloc[1:, 3:]
df = df.reindex(columns=[4, 5, 6, 7, 3])
df = df.apply(pd.to_numeric, errors='coerce')
df = df[df.iloc[:, -1] < 3]
df = df.sample(frac=1)

#### x y Werte zum Trainieren trennen

In [3]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]

#### Train / Test Daten trennen

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

#### Support Vector Machine trainieren

In [5]:
model = SVC(verbose=1, C=1000, gamma=0.002)
model.fit(x_train, y_train)

[LibSVM]

SVC(C=1000, gamma=0.002, verbose=1)

#### Mit Testdaten vergleichen

In [6]:
y_pred = model.predict(x_test)

#### Accuracy-, Precision-, Recall- und F1-Score ausgeben

In [7]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:\t{}\nPrecision:\t{}\nRecall:\t\t{}\nF1-Score:\t{}".format(accuracy, precision, recall, f1))

Accuracy:	0.48872180451127817
Precision:	0.42041754872618436
Recall:		0.48872180451127817
F1-Score:	0.4196083670441207


In [8]:
relImgPath = os.path.join("data", "images", "test")

imgPaths = []
for dirpath, dirnames, filenames in os.walk(relImgPath):
    if dirnames:
        classes = {}
        for index, name in enumerate(dirnames):
            classes[name]=index
    for filename in filenames:# [f for f in filenames if f.endswith(suportedImgFomats)]:
        imgPaths.append(os.path.join(dirpath, filename))

# TODO delete SPLITIT folder
print("Class names and indices:", classes)

features = ["Relative Image Path",
           "Class Name",
           "Class Index",
           "Aspect Ratio",
           "Number of Corners (Harris)",
           "Number of Corners (Shi-Tomasi)",
           "Perimeter Area Ratio"]

df = pd.DataFrame(columns=features)

for i, path in enumerate(imgPaths):
    c = path.split(os.sep)[-2]
    img = fe.prepared_image(path)
    ratio = fe.aspect_ratio(img)#path) 
    numCornersH = fe.num_corners(img, detector="harris")#fe.harris_corner_detection(img)#,path)#path)
    numCornersST = fe.num_corners(img, detector="shi-tomasi")#fe.shi_tomasi_corner_detection(img)#path)
    ratioPerim = fe.perimeter_area_ratio(img)
    
    row = pd.Series([path, c, classes[c], ratio, numCornersH, numCornersST, ratioPerim], index = features)
    df = df.append(row, ignore_index=True)
    
    for imgName in ["original", "prepared", "canny", "canny closed gaps", 
                    "contours", "max area contour", "harris", "shi-tomasi"]:
        fe.display_image(fe.images[imgName], title=imgName)
    
    print(str(format((100./len(imgPaths))*i, ".2f"))+" %", end="\r")


print("Information on the feature dataframe:")
df.info()

Class names and indices: {'bottleOpener': 0, 'canOpener': 1, 'corcScrew': 2, 'multiTool': 3}
